In [16]:
import pandas as pd

In [17]:
# Load the sample dataset
file_path = '../data/sampled_data.csv'
data = pd.read_csv(file_path)

In [18]:
print('There are {} rows in the dataset'.format(len(data)))

There are 1432660 rows in the dataset


In [19]:
data

,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,user_gender,user_id,artist_id,user_age,is_listened,ts_listen_year,ts_listen_month,release_year
0,7,1480241095,789250,92314,1,20051111,0,0,228,1,1,9620,177,26,0,2016,11,2005
1,77,1479052547,93477522,9495176,0,20150119,0,0,212,0,1,209,8135114,27,1,2016,11,2015
2,0,1479583581,120791740,12606392,0,20160311,0,0,236,0,0,4945,1435566,25,0,2016,11,2016
3,7,1479496255,67238735,6575789,0,20130517,0,0,367,0,0,2510,27,29,0,2016,11,2013
4,10,1479659993,910341,102632,11,20060925,2,1,218,0,0,5442,1172,21,1,2016,11,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432655,0,1479566555,125651234,13216890,0,20160603,0,0,214,0,1,7060,92773,22,0,2016,11,2016
1432656,297,1478281078,69686461,6830452,1,20130823,0,0,279,1,0,2348,12294,29,0,2016,11,2013
1432657,7,1479313227,4624032,425367,0,20091109,0,0,345,0,0,2109,106,21,1,2016,11,2009
1432658,7,1480209632,3099791,299887,0,20021231,0,0,227,0,1,3462,17236,21,1,2016,11,2002


To prepare data for matrix factorization using the Keras package, the dataset needs to be in a specific format that represents user-item interactions. The matrix factorization model typically expects the following format:

User-Item Interaction Matrix:
A matrix where rows represent users and columns represent items (e.g., songs), and the values represent the interaction between the user and the item (e.g., rating, play count, or a binary indicator if the item was played or skipped).


## Step 1: Importing Libraries

In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split #split dataset into training and testing sets.
from keras.models import Model #The core Keras class used to define the overall model.
from keras.layers import Input, Embedding, Flatten, Concatenate, Dense, Dropout #Keras layers used to build the neural network.
from keras.regularizers import l2 #Regularization technique to prevent overfitting by penalizing large weights.
import tensorflow as tf #Lower-level operations

In [21]:
print(data.dtypes)

genre_id           int64
ts_listen          int64
media_id           int64
album_id           int64
context_type       int64
release_date       int64
platform_name      int64
platform_family    int64
media_duration     int64
listen_type        int64
user_gender        int64
user_id            int64
artist_id          int64
user_age           int64
is_listened        int64
ts_listen_year     int64
ts_listen_month    int64
release_year       int64
dtype: object


In [ ]:
# Encoding categorical variables
data['user_id'] = data['user_id'].astype('category').cat.codes.values
data['media_id'] = data['media_id'].astype('category').cat.codes.values
data['album_id'] = data['album_id'].astype('category').cat.codes.values
data['artist_id'] = data['artist_id'].astype('category').cat.codes.values
data['genre_id'] = data['genre_id'].astype('category').cat.codes.values
data['platform_name'] = data['platform_name'].astype('category').cat.codes.values
data['platform_family'] = data['platform_family'].astype('category').cat.codes.values
data['context_type'] = data['context_type'].astype('category').cat.codes.values

In [13]:


# 
# # Splitting the data
# train, test = train_test_split(data, test_size=0.2, random_state=42)
# 
# # Defining input layers for all features
# user_input = Input(shape=(1,), name='user_input')
# media_input = Input(shape=(1,), name='media_input')
# album_input = Input(shape=(1,), name='album_input')
# artist_input = Input(shape=(1,), name='artist_input')
# genre_input = Input(shape=(1,), name='genre_input')
# user_gender_input = Input(shape=(1,), name='user_gender_input')
# user_age_input = Input(shape=(1,), name='user_age_input')
# platform_name_input = Input(shape=(1,), name='platform_name_input')
# platform_family_input = Input(shape=(1,), name='platform_family_input')
# context_type_input = Input(shape=(1,), name='context_type_input')
# media_duration_input = Input(shape=(1,), name='media_duration_input')
# release_date_input = Input(shape=(1,), name='release_date_input')
# listen_type_input = Input(shape=(1,), name='listen_type_input')
# 
# # Embedding layers for user and media, and other categorical variables with regularization
# user_embedding = Embedding(input_dim=data['user_id'].nunique(), output_dim=50, name='user_embedding', embeddings_regularizer=l2(0.01))(user_input)
# media_embedding = Embedding(input_dim=data['media_id'].nunique(), output_dim=50, name='media_embedding', embeddings_regularizer=l2(0.01))(media_input)
# album_embedding = Embedding(input_dim=data['album_id'].nunique(), output_dim=20, name='album_embedding', embeddings_regularizer=l2(0.01))(album_input)
# artist_embedding = Embedding(input_dim=data['artist_id'].nunique(), output_dim=30, name='artist_embedding', embeddings_regularizer=l2(0.01))(artist_input)
# genre_embedding = Embedding(input_dim=data['genre_id'].nunique(), output_dim=20, name='genre_embedding', embeddings_regularizer=l2(0.01))(genre_input)
# platform_name_embedding = Embedding(input_dim=data['platform_name'].nunique(), output_dim=10, name='platform_name_embedding', embeddings_regularizer=l2(0.01))(platform_name_input)
# platform_family_embedding = Embedding(input_dim=data['platform_family'].nunique(), output_dim=10, name='platform_family_embedding', embeddings_regularizer=l2(0.01))(platform_family_input)
# context_type_embedding = Embedding(input_dim=data['context_type'].nunique(), output_dim=10, name='context_type_embedding', embeddings_regularizer=l2(0.01))(context_type_input)
# 
# # Flatten embeddings
# user_vec = Flatten()(user_embedding)
# media_vec = Flatten()(media_embedding)
# album_vec = Flatten()(album_embedding)
# artist_vec = Flatten()(artist_embedding)
# genre_vec = Flatten()(genre_embedding)
# platform_name_vec = Flatten()(platform_name_embedding)
# platform_family_vec = Flatten()(platform_family_embedding)
# context_type_vec = Flatten()(context_type_embedding)
# 
# # Concatenate all features
# concat = Concatenate()([
#     user_vec, media_vec, album_vec, artist_vec, genre_vec, 
#     platform_name_vec, platform_family_vec, context_type_vec, 
#     user_gender_input, user_age_input, media_duration_input, 
#     release_date_input, listen_type_input
# ])
# 
# # Add dense layers with increased dropout and L2 regularization
# dense = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(concat)
# dense = Dropout(0.5)(dense)
# dense = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(dense)
# dense = Dropout(0.5)(dense)
# output = Dense(1, activation='sigmoid')(dense)
# 
# # Build and compile the model with a lower learning rate and gradient clipping
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# model = Model(inputs=[
#     user_input, media_input, album_input, artist_input, genre_input, 
#     user_gender_input, user_age_input, platform_name_input, platform_family_input, 
#     context_type_input, media_duration_input, release_date_input, listen_type_input
# ], outputs=output)
# model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
# 
# # Convert training data to NumPy arrays and ensure correct dtype
# X_train = [
#     np.array(train['user_id'], dtype=np.int32), np.array(train['media_id'], dtype=np.int32), 
#     np.array(train['album_id'], dtype=np.int32), np.array(train['artist_id'], dtype=np.int32), 
#     np.array(train['genre_id'], dtype=np.int32), np.array(train['user_gender'], dtype=np.int32), 
#     np.array(train['user_age'], dtype=np.float32), np.array(train['platform_name'], dtype=np.int32), 
#     np.array(train['platform_family'], dtype=np.int32), np.array(train['context_type'], dtype=np.int32), 
#     np.array(train['media_duration'], dtype=np.float32), np.array(train['release_date'], dtype=np.float32), 
#     np.array(train['listen_type'], dtype=np.int32)
# ]
# y_train = np.array(train['is_listened'], dtype=np.float32)
# 
# # Convert testing data to NumPy arrays and ensure correct dtype
# X_test = [
#     np.array(test['user_id'], dtype=np.int32), np.array(test['media_id'], dtype=np.int32), 
#     np.array(test['album_id'], dtype=np.int32), np.array(test['artist_id'], dtype=np.int32), 
#     np.array(test['genre_id'], dtype=np.int32), np.array(test['user_gender'], dtype=np.int32), 
#     np.array(test['user_age'], dtype=np.float32), np.array(test['platform_name'], dtype=np.int32), 
#     np.array(test['platform_family'], dtype=np.int32), np.array(test['context_type'], dtype=np.int32), 
#     np.array(test['media_duration'], dtype=np.float32), np.array(test['release_date'], dtype=np.float32), 
#     np.array(test['listen_type'], dtype=np.int32)
# ]
# y_test = np.array(test['is_listened'], dtype=np.float32)
# 
# # Train the model
# model.fit(X_train, y_train, epochs=5, batch_size=512, validation_data=(X_test, y_test))


Epoch 1/5
  52/2239 ━━━━━━━━━━━━━━━━━━━━ 6:20 174ms/step - accuracy: 0.5539 - loss: 558805.4375

KeyboardInterrupt: 

### 1. Accuracy:
- **Definition**: Accuracy is the percentage of correct predictions made by the model out of all predictions made.
- **Formula**:
$$
\text{Accuracy} = \frac{\text{Number of Correct Predictions}}{\text{Total Number of Predictions}} \times 100
$$

- **During Training**: This measures how well the model is performing on the training data, giving an indication of how much the model is learning from the training data.

### 2. Loss:
- **Definition**: Loss is a measure of how well the model’s predictions match the actual labels. It quantifies the difference between the predicted values and the actual values.
- **Common Loss Functions**:
  - **Binary Crossentropy**: Often used in binary classification problems (like predicting `0` or `1`).
  - **Mean Squared Error (MSE)**: Often used in regression problems.
- **During Training**: The loss is calculated after each forward pass of the model, and the model's weights are adjusted to minimize this loss. A lower loss indicates a better fit of the model to the data.

### 3. Validation Accuracy (val_accuracy):
- **Definition**: Validation accuracy is the accuracy calculated on a separate dataset (the validation set) that is not used for training but for monitoring the model’s performance during training.
- **Purpose**: This metric helps in assessing how well the model generalizes to new, unseen data. It's a key indicator of whether the model is overfitting (performing well on the training data but poorly on unseen data).

### 4. Validation Loss (val_loss):
- **Definition**: Validation loss is the loss calculated on the validation dataset. Like validation accuracy, it is used to monitor the model's performance on unseen data.
- **Purpose**: A decreasing validation loss indicates that the model is generalizing well to new data. If the validation loss starts to increase while the training loss continues to decrease, it may be a sign of overfitting.
